<h1>Introduction to <i style="color:red;">SQLAlchemy</i></h1>

In [1]:
%%HTML
<img src="sqla_logo.png" width="300"/>

## <i>by Jack Camier, Python Developer</i>

SQLAlchemy is well documented at: https://www.sqlalchemy.org/

<h2>created by <i style="color:blue;">Mike Bayer</i></h2>

https://github.com/zzzeek

## There are a lot of tutorials on the subject, honestly too many to count.
## I hope to provide a tutorial that provides, some cookbook recipes (code snippets) and a little bit more visualization via a Jupyter Notebook to help developers better grasp the concepts than what I have found on the internet
<hr>

## "SQLAlchemy is the Python SQL toolkit and Object Relational Mapper that gives application developers the full power and flexibility of SQL" - Mike Bayer
https://www.sqlalchemy.org/
<hr>

## Our tutorial today will be using SQLite

In [2]:
%%HTML
<img src="sqlite_logo.gif" width="200"/>

https://www.sqlite.org/index.html

## The reason being you won't have to download or install anything in regards to a database perspective to do this tutorial 
## Also, `sqlite3` comes built-in with standard `Cpython`

In [3]:
import sqlite3 # That's it!

## "SQLite is a C-language library that implements a small, fast, self-contained, high-reliability, full-featured, SQL database engine. SQLite is the most used database engine in the world. SQLite is built into all mobile phones and most computers and comes bundled inside countless other applications that people use every day."
https://www.sqlite.org/index.html

## Now you could do things like the following to work with sqlite using the standard library `sqlite3` such as:

In [4]:
import uuid

sql_values = {
"id": str(uuid.uuid4()),
"first_name": "Bill",
"last_name": "Smith",
"email": "b.smith@mail.com"
}

sql_values

{'id': 'b54360fe-7a28-4db4-b7d9-c22290d63380',
 'first_name': 'Bill',
 'last_name': 'Smith',
 'email': 'b.smith@mail.com'}

In [5]:
# Create connection session and database
conn = sqlite3.connect('example.db')

# Create cursor and assign it to `c`
c = conn.cursor()

# Create table using raw SQL
c.execute('''CREATE TABLE users (id text, first_name text, last_name text, email text)''')

In [6]:
# Insert a row of data 
c.execute('INSERT INTO users VALUES (:id, :first_name, :last_name, :email)', sql_values)
conn.commit()

In [7]:
# Select the data and print out row results
sql = c.execute('''SELECT * from users''')
for row in sql:
    print(row)

('b54360fe-7a28-4db4-b7d9-c22290d63380', 'Bill', 'Smith', 'b.smith@mail.com')


In [8]:
# Remove temporary `example.db` database
import os
os.remove("example.db")

## For simple things like this, we probably don't need SQLAlchemy. But normally within a short period of time, requirments of a project or feautures of a product change rapidly and we need to do something more like:

In [9]:
"""
select 
  'today - ' || to_char(trunc(sysdate),'Mon FMDDFM'),
  trunc(sysdate) as deadline
from dual 
UNION
select 
  'tomorrow - '|| to_char(trunc(sysdate+1),'Mon FMDDFM'), 
  trunc(sysdate+1) as deadline
from dual 
UNION
select
  'next week - '|| to_char(trunc(sysdate+7),'Mon FMDDFM'), 
  trunc(sysdate+7) as deadline
from dual 
UNION
select 
  'next month - '|| to_char(trunc(ADD_MONTHS(sysdate,1)),'Mon FMDDFM'), 
  trunc(ADD_MONTHS(sysdate,1)) as deadline 
from dual
UNION
select 
  name || ' - ' || to_char(deadline, 'Mon FMDDFM'), 
  deadline
from ticket_deadlines 
where project_id = :project_id
and deadline >= trunc(sysdate)
order by deadline
"""
# https://philip.greenspun.com/sql/complex-queries.html

"\nselect \n  'today - ' || to_char(trunc(sysdate),'Mon FMDDFM'),\n  trunc(sysdate) as deadline\nfrom dual \nUNION\nselect \n  'tomorrow - '|| to_char(trunc(sysdate+1),'Mon FMDDFM'), \n  trunc(sysdate+1) as deadline\nfrom dual \nUNION\nselect\n  'next week - '|| to_char(trunc(sysdate+7),'Mon FMDDFM'), \n  trunc(sysdate+7) as deadline\nfrom dual \nUNION\nselect \n  'next month - '|| to_char(trunc(ADD_MONTHS(sysdate,1)),'Mon FMDDFM'), \n  trunc(ADD_MONTHS(sysdate,1)) as deadline \nfrom dual\nUNION\nselect \n  name || ' - ' || to_char(deadline, 'Mon FMDDFM'), \n  deadline\nfrom ticket_deadlines \nwhere project_id = :project_id\nand deadline >= trunc(sysdate)\norder by deadline\n"

## "I wrote a reporting query once that was 700 lines long and visited 27 different tables in lookups or joins" - Greg Kemnitz
https://www.quora.com/Whats-the-most-complex-SQL-query-you-ever-wrote

## Figuring out how to do this in raw SQL can be very challenging if you are not a DBA or well versed in SQL syntax.
## With SQLAlchemy we can use the abstraction layer and write python code we are more familiar with and let the toolkit do the SQL magic behind the scenes

## First there are two main components of SQLAlchemy:
## "SQLAlchemy consists of two distinct components, known as the <i style="color:red;">Core</i> and the <i style="color:red;">ORM</i>. 
## The Core is itself a fully featured SQL abstraction toolkit, providing a smooth layer of abstraction over a wide variety of DBAPI implementations and behaviors, as well as a SQL Expression Language which allows expression of the SQL language via generative Python expressions."

## "The Object Relational Mapper (ORM) is then an optional package which builds upon the Core."
https://www.sqlalchemy.org/features.html

In [10]:
%%HTML
<img src="sqla_arch_small.png" width="500"/>

https://docs.sqlalchemy.org/en/13/intro.html

## To get sqlalchemy installed, all you have to do is:
`pip install SQLAlchemy`
## Then

In [11]:
import sqlalchemy

## In this talk, I am using version:

In [12]:
sqlalchemy.__version__

'1.3.9'

# To get started, we will commence looking at the Core component of SQLAlchemy

## Please note that individual database flavors, also called `dialects`, enumerate the available DBAPIs for each database
## Most seasoned developers learn that each database has its own nuances and slight variations of SQL syntax depending on the database. 
## Here are a list of popular `dialects` for SQLAlchemy:

# - PostgreSQL
https://docs.sqlalchemy.org/en/13/dialects/postgresql.html
# - MySQL
https://docs.sqlalchemy.org/en/13/dialects/mysql.html
# - SQLite
https://docs.sqlalchemy.org/en/13/dialects/sqlite.html
# - Oracle
https://docs.sqlalchemy.org/en/13/dialects/oracle.html
# - Microsoft SQL Server
https://docs.sqlalchemy.org/en/13/dialects/mssql.html
<hr>

## Since our focus will be SQLite, here are the main data type affinities:
* TEXT
* NUMERIC
* INTEGER
* REAL
* BLOB

https://www.sqlite.org/datatype3.html#affinity
<hr>

## "The SQLAlchemy Expression Language presents a system of representing relational database structures and expressions using Python constructs. These constructs are modeled to resemble those of the underlying database as closely as possible, while providing a modicum of abstraction of the various implementation differences between database backends."
https://docs.sqlalchemy.org/en/13/core/tutorial.html

##  <i style="color:red;">Huh!?</i> Translation:
## `SQLAlchemy Expression Language` is a way to represent everything dealing with a database using Python. 

## For this tutorial we will use an in-memory-only SQLite database. 

In [13]:
from sqlalchemy import create_engine

# To make the connection to the sqlite database
engine = create_engine('sqlite:///:memory:', echo=True)


## The `echo` flag is a shortcut to setting up SQLAlchemy logging using Python’s standard logging module.
## With it enabled, we’ll see all the generated SQL scripts produced. I believe this is very helpful to see visually what is being done behind the scenes

## "The `engine` is the starting point for any SQLAlchemy application. It’s “home base” for the actual database and its DBAPI, delivered to the SQLAlchemy application through a connection pool and a Dialect..."
https://docs.sqlalchemy.org/en/13/core/engines.html

In [14]:
%%HTML
<img src="sqla_engine_arch.png" width="700"/>

## To use the `create_engine` method to an actual sqlite database, here are some examples:

In [15]:
# where <path> is relative:
engine = create_engine('sqlite:///foo.db')

# Linux/Mac - 4 initial slashes in total
engine = create_engine('sqlite:////absolute/path/to/foo.db')

# Windows
engine = create_engine('sqlite:///C:\\path\\to\\foo.db')

In [16]:
## I personally like to do something more like:

PROJECT_DIR = os.path.dirname(os.path.abspath(''))

# Note you can also do:

# os.path.dirname(os.path.abspath(__file__))

# But this only works with a python .py file, not in Jupyter notebook .ipynb

PROJECT_DIR

'/Users/jacquescamier/JupyterProjects'

## Note: the file path will be different for you since `os.path.dirname` etc. is specific to your computer

In [17]:
database_path = os.path.join(PROJECT_DIR, 'your_database.db')
sqlite_path = f"sqlite:///{database_path}"

sqlite_path

'sqlite:////Users/jacquescamier/JupyterProjects/your_database.db'

In [18]:
engine = create_engine(sqlite_path)

engine

Engine(sqlite:////Users/jacquescamier/JupyterProjects/your_database.db)

## Let's go back to our in-memory example

In [19]:
engine = create_engine('sqlite:///:memory:', echo=True)

## Now let's import the necessary objects to build a table

In [20]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey, Boolean

## A collection of metadata entities is stored in an object called MetaData
## "MetaData is a container object that keeps together many different features of a database (or multiple databases) being described"
https://docs.sqlalchemy.org/en/13/core/metadata.html#metadata-describing

In [21]:
# Instantiate the MetaData object
metadata = MetaData()

In [22]:
# Define the `users` table:
users = Table('users', metadata,
              Column('id', Integer, primary_key=True),
              Column('first_name', String),
              Column('last_name', String),
             )

# Define the `email` table:
email = Table('email', metadata,
                 Column('id', Integer, primary_key=True),
                 Column('user_id', None, ForeignKey('users.id')),
                 Column('email_address', String, nullable=False),
                 Column('is_active', Boolean, default=True)
                 )

In [23]:
# Create the tables:
metadata.create_all(engine)

2020-04-13 18:13:49,642 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-04-13 18:13:49,643 INFO sqlalchemy.engine.base.Engine ()
2020-04-13 18:13:49,644 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-04-13 18:13:49,645 INFO sqlalchemy.engine.base.Engine ()
2020-04-13 18:13:49,647 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2020-04-13 18:13:49,647 INFO sqlalchemy.engine.base.Engine ()
2020-04-13 18:13:49,648 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2020-04-13 18:13:49,649 INFO sqlalchemy.engine.base.Engine ()
2020-04-13 18:13:49,651 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("email")
2020-04-13 18:13:49,651 INFO sqlalchemy.engine.base.Engine ()
2020-04-13 18:13:49,652 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("email")
2020-04-13 18:13:49,653 INFO sqlalchemy.engine.base.Engine ()
2020-04-13 18:13:49,654 IN

## ^ As you can see SQLAlchemy did a lot behind the scenes from our few lines of Python code

## Let's insert some data into our table

In [24]:
ins = users.insert().values(first_name='Bill', last_name='Smith')

## To see the insert SQL statement, we can use the str() function

In [25]:
str(ins)

'INSERT INTO users (first_name, last_name) VALUES (:first_name, :last_name)'

## To make a connection to the database, we use the `connect` method

In [26]:
conn = engine.connect()

## Now, let's pass the our assigned SQL insert `ins` to the connection
## This is where it actual writes to the database

In [27]:
result = conn.execute(ins)

2020-04-13 18:13:52,274 INFO sqlalchemy.engine.base.Engine INSERT INTO users (first_name, last_name) VALUES (?, ?)
2020-04-13 18:13:52,275 INFO sqlalchemy.engine.base.Engine ('Bill', 'Smith')
2020-04-13 18:13:52,277 INFO sqlalchemy.engine.base.Engine COMMIT


## Let's insert another record in the database

In [28]:
ins = users.insert().values(first_name='Sally', last_name='Jackson')

In [29]:
result = conn.execute(ins)

2020-04-13 18:13:53,211 INFO sqlalchemy.engine.base.Engine INSERT INTO users (first_name, last_name) VALUES (?, ?)
2020-04-13 18:13:53,212 INFO sqlalchemy.engine.base.Engine ('Sally', 'Jackson')
2020-04-13 18:13:53,214 INFO sqlalchemy.engine.base.Engine COMMIT


## Now let's select the data from the database

In [30]:
from sqlalchemy.sql import select

select_users = select([users])

result = conn.execute(select_users)

2020-04-13 18:13:53,640 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.first_name, users.last_name 
FROM users
2020-04-13 18:13:53,641 INFO sqlalchemy.engine.base.Engine ()


In [31]:
# Iterate through the results:

for row in result:
    print(row)

(1, 'Bill', 'Smith')
(2, 'Sally', 'Jackson')


## In most cases, you are going to want to insert several records at a time by passing a list dictionaries:

In [32]:
# You can pass a list of dictionaries
conn.execute(email.insert(), [
    {'user_id': 1, 'email_address' : 'billsmith@somemail.com'},
    {'user_id': 2, 'email_address' : 's.jackson@somemail.com'},
    ])

2020-04-13 18:13:54,359 INFO sqlalchemy.engine.base.Engine INSERT INTO email (user_id, email_address, is_active) VALUES (?, ?, ?)
2020-04-13 18:13:54,360 INFO sqlalchemy.engine.base.Engine ((1, 'billsmith@somemail.com', 1), (2, 's.jackson@somemail.com', 1))
2020-04-13 18:13:54,363 INFO sqlalchemy.engine.base.Engine COMMIT


# To do a select statement to get one record:

In [33]:
result = conn.execute(select_users)

row = result.fetchone()

2020-04-13 18:13:54,834 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.first_name, users.last_name 
FROM users
2020-04-13 18:13:54,836 INFO sqlalchemy.engine.base.Engine ()


In [34]:
print(row)

(1, 'Bill', 'Smith')


## Using a list comprehension:

In [35]:
list_rows = [row for row in conn.execute(select_users)]

2020-04-13 18:13:55,498 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.first_name, users.last_name 
FROM users
2020-04-13 18:13:55,499 INFO sqlalchemy.engine.base.Engine ()


In [36]:
# Print first record
list_rows[0]

(1, 'Bill', 'Smith')

# You can get specific `columns` by using the `c` attribute

In [37]:
select_first_names = select([users.c.first_name])

[row for row in conn.execute(select_first_names)]

2020-04-13 18:13:56,186 INFO sqlalchemy.engine.base.Engine SELECT users.first_name 
FROM users
2020-04-13 18:13:56,187 INFO sqlalchemy.engine.base.Engine ()


[('Bill',), ('Sally',)]

# We can do some unpacking magic and get rid of the tuple, notice the `,` after the `for row` expression

In [38]:
[row for row, in conn.execute(select_first_names)]

2020-04-13 18:13:56,644 INFO sqlalchemy.engine.base.Engine SELECT users.first_name 
FROM users
2020-04-13 18:13:56,645 INFO sqlalchemy.engine.base.Engine ()


['Bill', 'Sally']

# Please recall that we declared a `ForeignKey` relationship between the users and email tables:
`Column('user_id', None, ForeignKey('users.id'))`

## This sets us up to do joins such as:

In [39]:
select_join_user_and_email = select([users, email]).where(users.c.id == email.c.user_id)

[row for row in conn.execute(select_join_user_and_email)]

2020-04-13 18:13:57,389 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.first_name, users.last_name, email.id, email.user_id, email.email_address, email.is_active 
FROM users, email 
WHERE users.id = email.user_id
2020-04-13 18:13:57,390 INFO sqlalchemy.engine.base.Engine ()


[(1, 'Bill', 'Smith', 1, 1, 'billsmith@somemail.com', True),
 (2, 'Sally', 'Jackson', 2, 2, 's.jackson@somemail.com', True)]

## Here is another way to do joins, albeit a bit more complicated

In [40]:
from sqlalchemy import join

j = users.join(email, users.c.id == email.c.user_id)
select_users_email_join = select([users, email]).select_from(j)

[row for row in conn.execute(select_users_email_join)]

2020-04-13 18:13:57,825 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.first_name, users.last_name, email.id, email.user_id, email.email_address, email.is_active 
FROM users JOIN email ON users.id = email.user_id
2020-04-13 18:13:57,826 INFO sqlalchemy.engine.base.Engine ()


[(1, 'Bill', 'Smith', 1, 1, 'billsmith@somemail.com', True),
 (2, 'Sally', 'Jackson', 2, 2, 's.jackson@somemail.com', True)]

## Now Let's load some data from a csv file and write to the database

In [100]:
import csv

with open('users_data.csv', mode='r') as user_file:
    reader = csv.reader(user_file, delimiter=',')
    headers = next(reader, None)
    import_list = [{headers[0]: row[0], headers[1]: row[1]} for idx, row in enumerate(reader)]
    

import_list

[{'first_name': 'Jerome', 'last_name': 'Walters'},
 {'first_name': 'Rick', 'last_name': 'Stein'},
 {'first_name': 'Sam', 'last_name': 'West'},
 {'first_name': 'Jill', 'last_name': 'Smith'},
 {'first_name': 'Jan', 'last_name': 'West'},
 {'first_name': 'Linda', 'last_name': 'Rodriguez'}]

In [101]:
# Insert rows to database
conn.execute(users.insert(), import_list)

2020-04-13 20:22:05,032 INFO sqlalchemy.engine.base.Engine INSERT INTO users (first_name, last_name) VALUES (?, ?)
2020-04-13 20:22:05,034 INFO sqlalchemy.engine.base.Engine (('Jerome', 'Walters'), ('Rick', 'Stein'), ('Sam', 'West'), ('Jill', 'Smith'), ('Jan', 'West'), ('Linda', 'Rodriguez'))
2020-04-13 20:22:05,036 INFO sqlalchemy.engine.base.Engine COMMIT


In [103]:
# Select all users
result = conn.execute(select([users]))

[row for row in result]

2020-04-13 20:24:54,800 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.first_name, users.last_name 
FROM users
2020-04-13 20:24:54,801 INFO sqlalchemy.engine.base.Engine ()


[(1, 'Bill', 'Smith'),
 (2, 'Sally', 'Jackson'),
 (3, 'Jerome', 'Walters'),
 (4, 'Rick', 'Stein'),
 (5, 'Sam', 'West'),
 (6, 'Jill', 'Smith'),
 (7, 'Jan', 'West'),
 (8, 'Linda', 'Rodriguez')]

## Select all users where last name is Smith

In [111]:
s = select([(users.c.last_name + ", " + users.c.first_name)]).where(users.c.last_name=="Smith")

conn.execute(s).fetchall()

2020-04-13 20:39:46,047 INFO sqlalchemy.engine.base.Engine SELECT users.last_name || ? || users.first_name AS anon_1 
FROM users 
WHERE users.last_name = ?
2020-04-13 20:39:46,048 INFO sqlalchemy.engine.base.Engine (', ', 'Smith')


[('Smith, Bill',), ('Smith, Jill',)]